In [1]:
import sys
import os
import subprocess
import tarfile
import shutil
import math
import time
import random
import subprocess
import tempfile

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d
from Bio import motifs
import pickle
from datetime import datetime
from tqdm import tqdm

import boda
from boda.generator.parameters import StraightThroughParameters
from boda.generator import AdaLead
from boda.generator.energy import OverMaxEnergy, StremePenalty
from boda.common import constants, utils
from boda.generator.plot_tools import matrix_to_dms, ppm_to_IC, ppm_to_pwm

boda_src = os.path.join( os.path.dirname( os.path.dirname( os.getcwd() ) ), 'src' )
sys.path.insert(0, boda_src)

from main import unpack_artifact, model_fn
from pymeme import streme, parse_streme_output

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [23]:
all_pickle_paths = !gsutil ls $'gs://syrgoth/boda_library_design_202112/pickles'
all_pickle_paths = all_pickle_paths[1:]
all_pickle_paths

['gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_002__round_0__20211206_025443__72806280.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_002__round_1__20211206_025528__41332638.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_002__round_2__20211206_025555__20655304.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_002__round_3__20211206_025633__84440783.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_002__round_4__20211206_025708__20020621.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_002__round_5__20211206_025737__87493515.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_005__round_0__20211206_024503__33072627.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_009__round_0__20211208_015040__72147769.pt',
 'gs://syrgoth/boda_library_design_202112/pickles/rc__hepg2__al__track_0

In [24]:
all_proposal_dicts = {}
for file_name in tqdm(all_pickle_paths):
    with tempfile.TemporaryDirectory() as tmpdir:
        temp_loc = os.path.join(tmpdir, os.path.basename(file_name)) 
        subprocess.check_call(['gsutil', 'cp', file_name, temp_loc])
        all_proposal_dicts[os.path.basename(file_name).rstrip('.pt')] = torch.load(temp_loc)

100%|██████████| 65/65 [01:28<00:00,  1.36s/it]


In [20]:
for key in all_proposal_dicts.keys():
    user, celltype, generator, track_num, round_num, _, _ = key.split('__')
    if user == 'sg':
        proposals_list = all_proposal_dicts[key]['proposals']
        for round_num, proposal_round in enumerate(proposals_list):
            print(round_num)

['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'track_005', 'round_5', '20211207_185623', '627393']
['sg', 'sknsh', 'sa', 'tr

In [28]:
len(all_proposal_dicts[key]['proposals'])

6